In [17]:

from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T


import os
import sys
import pickle
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss, confusion_matrix
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'image_textvsnontext/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['level_2', 'textVsNonText.ipynb']


In [7]:
def read_all(folder_path, key_prefix=""):
    print("Reading:")
    images = {}
    files = os.listdir(folder_path)
    for i, file_name in tqdm_notebook(enumerate(files), total=len(files)):
        file_path = os.path.join(folder_path, file_name)
        image_index = key_prefix + file_name[:-4]
        image = Image.open(file_path)
        image = image.convert("L")
        images[image_index] = np.array(image.copy()).flatten()
        image.close()
    return images


In [8]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [15]:
LEVEL = 'level_2'

In [18]:
languages = ['ta', 'hi', 'en']

images_train = read_all("./drive/MyDrive/image_textvsnontext/level_2/"+"background", key_prefix='bgr_') # change the path
for language in languages:
  images_train.update(read_all("./drive/MyDrive/image_textvsnontext/level_2/"+language, key_prefix=language+"_" ))
print(len(images_train))

# images_test = read_all("./drive/MyDrive/image_textvsnontext/level_2/", key_prefix='') # change the path
# print(len(images_test))

Reading:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Reading:



Reading:



Reading:



900


In [21]:
X_train = []
Y_train = []
for key, value in images_train.items():
    X_train.append(value)
    if key[:4] == "bgr_":
        Y_train.append(0)
    else:
        Y_train.append(1)



In [23]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)



In [29]:
print("Train size : ", X_scaled_train.shape)
Y_train= np.array(Y_train)
print("labels size : ",Y_train.shape)

Train size :  (900, 256)
labels size :  (900,)


In [31]:
!pip install git+https://github.com/deepvision-class/starter-code
import coutils


  Cloning https://github.com/deepvision-class/starter-code to /tmp/pip-req-build-uvidcoci
  Running command git clone -q https://github.com/deepvision-class/starter-code /tmp/pip-req-build-uvidcoci
  Created wheel for Colab-Utils: filename=Colab_Utils-0.1.dev0-cp36-none-any.whl size=10324 sha256=70e742f4c250741e391b78ac44086aedd3981ab64a424c75af82815e4ddfbd59
  Stored in directory: /tmp/pip-ephem-wheel-cache-q28gy6qc/wheels/63/d1/27/a208931527abb98d326d00209f46c80c9d745851d6a1defd10
Successfully built Colab-Utils


In [32]:
from coutils import Solver

In [33]:
dtype = torch.float
ltype = torch.long

if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

print_every = 100

print('using device:', device)

using device: cpu


In [90]:
def check_accuracy(X_train,Y_train, model):

  num_correct = 0
  num_samples = 0
  model.eval() 
  num_train = X_train.shape[0]
  batch_size=10
  with torch.no_grad():
    idxs = torch.randint(num_train,(batch_size,))
    X_batch = X_train[idxs]
    y_batch = Y_train[idxs]
    x = X_batch.to(device=device, dtype=dtype)  
    y = y_batch.to(device=device, dtype=ltype)
    scores = model(x)
    _, preds = scores.max(1)
    num_correct += (preds == y).sum()
    num_samples += preds.size(0)
  acc = float(num_correct) / num_samples
  print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
  return acc

In [92]:
def adjust_learning_rate(optimizer, lrd, epoch, schedule):

  if epoch in schedule:
    for param_group in optimizer.param_groups:
      print('lr decay from {} to {}'.format(param_group['lr'], param_group['lr'] * lrd))
      param_group['lr'] *= lrd

def train_model(model, optimizer, X_train,Y_train,epochs=1, learning_rate_decay=.1, schedule=[], verbose=True):

  model = model.to(device=device)  # move the model parameters to CPU/GPU
  num_iters = epochs * 10
  if verbose:
    num_prints = num_iters // print_every + 1
  else:
    num_prints = epochs
  acc_history = torch.zeros(num_prints, dtype=torch.float)
  iter_history = torch.zeros(num_prints, dtype=torch.long)
  for e in range(epochs):
    
    adjust_learning_rate(optimizer, learning_rate_decay, e, schedule)
    num_train = X_train.shape[0]
    batch_size=32
    for i in range(10):
      model.train() 
      idxs = torch.randint(num_train,(batch_size,))
      X_batch = X_train[idxs]
      y_batch = Y_train[idxs]
      x = X_batch.to(device=device, dtype=dtype)  
      y = y_batch.to(device=device, dtype=ltype)
      scores = model(x)
      loss = F.cross_entropy(scores, y)

      optimizer.zero_grad()

      loss.backward()


      optimizer.step()

      tt = t + e * 10

      if verbose and (tt % print_every == 0 or (e == epochs-1 and t == len(X_train)-1)):
        print('Epoch %d, Iteration %d, loss = %.4f' % (e, tt, loss.item()))
        acc = check_accuracy(X_train,Y_train, model)
        acc_history[tt // print_every] = acc
        iter_history[tt // print_every] = tt
        print()

  return acc_history, iter_history
def flatten(x, start_dim=1, end_dim=-1):
  return x.flatten(start_dim=start_dim, end_dim=end_dim)
class Flatten(nn.Module):
  def forward(self, x):
    return flatten(x)

In [93]:
from torch.utils.data import DataLoader
from torch.utils.data import sampler


In [66]:
X_scaled_train = torch.tensor(X_scaled_train).reshape(-1,1,16,16)
Y_train = torch.tensor(Y_train)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [104]:
model

TextVNonText(
  (model): Sequential(
    (0): Conv2d(1, 78, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=1, stride=4, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(78, 216, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=864, out_features=2, bias=True)
  )
)

In [103]:
class TextVNonText(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=78,kernel_size=3,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1,stride=4),
        nn.Conv2d(in_channels=78,out_channels=216,stride=1,kernel_size=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1,stride=2),
        nn.Flatten(start_dim=1,end_dim=-1),
        nn.Linear(in_features=216*2*2,out_features=2)
      
        )
    
  
  def forward(self, x):
    scores = None
    scores = self.model(x)
    return scores

learning_rate = 1e-3
momentum = 0.5
weight_decay = 1e-4
model = TextVNonText(10)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
_ = train_model(model, optimizer,X_scaled_train,Y_train,epochs=20)

Epoch 0, Iteration 0, loss = 0.6903
Got 9 / 10 correct (90.00)

Epoch 0, Iteration 0, loss = 0.6364
Got 8 / 10 correct (80.00)

Epoch 0, Iteration 0, loss = 0.6322
Got 10 / 10 correct (100.00)

Epoch 0, Iteration 0, loss = 0.5674
Got 8 / 10 correct (80.00)

Epoch 0, Iteration 0, loss = 0.5387
Got 6 / 10 correct (60.00)

Epoch 0, Iteration 0, loss = 0.5368
Got 8 / 10 correct (80.00)

Epoch 0, Iteration 0, loss = 0.5268
Got 8 / 10 correct (80.00)

Epoch 0, Iteration 0, loss = 0.4715
Got 8 / 10 correct (80.00)

Epoch 0, Iteration 0, loss = 0.5047
Got 9 / 10 correct (90.00)

Epoch 0, Iteration 0, loss = 0.4450
Got 10 / 10 correct (100.00)

Epoch 10, Iteration 100, loss = 0.0114
Got 10 / 10 correct (100.00)

Epoch 10, Iteration 100, loss = 0.0418
Got 10 / 10 correct (100.00)

Epoch 10, Iteration 100, loss = 0.1961
Got 10 / 10 correct (100.00)

Epoch 10, Iteration 100, loss = 0.0336
Got 10 / 10 correct (100.00)

Epoch 10, Iteration 100, loss = 0.0194
Got 10 / 10 correct (100.00)

Epoch 10, I